# Project: Deep Learning - Pytorch

**Instructions for Students:**

Please carefully follow these steps to complete and submit your project:

1. **Make a copy of the Project**: Please make a copy of this project either to your own Google Drive or download locally. Work on the copy of the project. The master project is **Read-Only**, meaning you can edit, but it will not be saved when you close the master project. To avoid total loss of your work, remember to make a copy.

2. **Completing the Project**: You are required to work on and complete all tasks in the provided project. Be disciplined and ensure that you thoroughly engage with each task.
   
3. **Creating a Google Drive Folder**: Each of you must create a new folder on your Google Drive. This will be the repository for all your completed project files, aiding you in keeping your work organized and accessible.
   
4. **Uploading Completed Project**: Upon completion of your project, make sure to upload all necessary files, involving codes, reports, and related documents into the created Google Drive folder. Save this link in the 'Student Identity' section and also provide it as the last parameter in the `submit` function that has been provided.
   
5. **Sharing Folder Link**: You're required to share the link to your project Google Drive folder. This is crucial for the submission and evaluation of your project.
   
6. **Setting Permission to Public**: Please make sure your Google Drive folder is set to public. This allows your instructor to access your solutions and assess your work correctly.

Adhering to these procedures will facilitate a smooth project evaluation process for you and the reviewers.

## Project Description

The Deep Learning Projects are divided into two parts, the first is the Calculations worth 30% and the second one is Pytorch Project worth 70% in this notebook.

The two projects will help you gain experience to learn about Deep Learning in detail.

In this project, you will use what you learn to create your own Deep Learning model. We'll use a variety of datasets, each with different data types such as images, text, and numerical/categorical data. Your task is to choose one of these datasets and build a deep learning model upon it.


## Datasets

Choose one from the following datasets for your project:

1. **Digits Dataset**: A simplified version of MNIST containing 8x8 images of hand-written digits. [Dataset Link](https://scikit-learn.org/stable/auto_examples/datasets/plot_digits_last_image.html)
2. **Dogs vs. Cats Dataset from Kaggle**: A sizable dataset of 25,000 images with equal representation of dogs and cats. [Dataset Link](https://www.kaggle.com/c/dogs-vs-cats)
3. **Breast Cancer Wisconsin (Diagnostic) Dataset**: Computed features from a digitized image of a fine needle aspirate (FNA) of a breast mass. [Dataset Link](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)
4. **Spam Text Message Classification Dataset**: A collection of labeled SMS messages, categorized as "spam" or "ham". [Dataset Link](https://www.kaggle.com/uciml/sms-spam-collection-dataset)
5. **German Credit Risk Dataset**: People's data categorized by various attributes to predict credit risk. [Dataset Link](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29)



## Grading Criteria

Your work will be evaluated based on both accuracy and loss value:

100: The model has an accuracy of more than 80% and a Loss Value of less than 0.2. This model is excellent and demonstrates a strong understanding of the task.

90: The model has an accuracy between 70% - 79% and a Loss Value between 0.2 - 0.3. This model is very good, with some room for improvement.

80: The model has an accuracy between 60% - 69% and a Loss Value between 0.3 - 0.4. This model is fairly good but needs improvement in balancing accuracy and loss value.

70: The model has an accuracy between 50% - 59% and a Loss Value between 0.4 - 0.5. This model is below average and needs significant improvement.

60 or below: The model has an accuracy of less than 50% or a Loss Value of more than 0.5, or the student did not submit the accuracy and Loss Value. This model is poor and needs considerable improvement.

Rmember to make a copy of this notebook in your Google Drive and work in your own copy.

Let's start your deep learning journey! Choose your dataset and delve into the project! Happy modeling!

## Student Identity

In [8]:
# @title #### Student Identity
student_id = "REAPQGKN" # @param {type:"string"}
name = "Diwa Prasetyo" # @param {type:"string"}
drive_link = "https://drive.google.com/drive/folders/15QW1n6YhblOOLjsfIUwjNu5ksSKFo6kJ?usp=drive_link"  # @param {type:"string"}
assignment_id = "00_pytorch_project"

# Import grader package
!pip install rggrader
from rggrader import submit, submit_image

## Project Structure

Your project should be organized into five main sections.

### 1. Package and Module Installation

First, let's pool all package and module that you'll need in the installation section below.

In [9]:
# !pip install torch torchvision torchaudio torchtext scikit-learn pandas numpy matplotlib


In [10]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torchtext

Looking in indexes: https://download.pytorch.org/whl/cu118


### 2. Data Loading and Preprocessing

Load the chosen dataset and preprocess it for deep learning.

In [11]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Tokenization
def tokenizer(text):
    return word_tokenize(text.lower())

# Load your data
df = pd.read_csv('spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'text']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
import re
import string
def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
  text = re.sub(r'#[A-Za-z0-9]+', '', text) # Menghapus hastag
  text = re.sub(r'RT[\s]+', '', text) # Menghapus RT
  text = re.sub(r'https?:\/\/\S+', '', text) # Menghapus tautan
  text = re.sub(r'[^\w\s]', '', text) # Menghapus karakter selain huruf dan angka
  text = text.replace('\n', ' ') # Mengganti karakter newline dengan spasi
  text = text.strip() # Menghapus spasi ekstra di awal dan akhir teks
  text = text.translate(str.maketrans('', '', string.punctuation)) # Menghapus semua tanda baca
  return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
  text = text.lower()
  return text


In [13]:
# Build vocabulary
vocab = set()
for sentence in df['text']:
    tokens = tokenizer(sentence)
    vocab.update(tokens)

vocab = {word: idx for idx, word in enumerate(vocab, start=1)}

# Convert text to tensor
def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text) if token in vocab]

# Apply text_pipeline on dataset
df['text'] = df['text'].apply(text_pipeline)

# Split the dataset into training and validation
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42)

# Pad sequences to the same length
def collate_batch(batch):
    text_list, label_list = [], []
    for (text, label) in batch:
        processed_text = torch.tensor(text, dtype=torch.int64)
        text_list.append(processed_text)
        label_list.append(torch.tensor(label, dtype=torch.int64))
    return pad_sequence(text_list, batch_first=True), torch.tensor(label_list)

# Create DataLoader for training and testing
train_data = list(zip(train_texts, train_labels))
test_data = list(zip(test_texts, test_labels))

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_data, batch_size=32, collate_fn=collate_batch)

### 3. Model Building

Define your deep learning model's architecture.

In [14]:
# Write your code here for Model Building here
import torch.nn as nn

class SpamClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(SpamClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        rnn_out, _ = self.rnn(embedded)
        final_feature_map = rnn_out[:, -1, :]  # get the output of the last time step
        out = self.fc(final_feature_map)
        return out

# Hyperparameters
vocab_size = len(vocab) + 1
embed_dim = 256
hidden_dim = 128
output_dim = 2

model = SpamClassifier(vocab_size, embed_dim, hidden_dim, output_dim)

# define your Deep Learning Model here, training is in the next section.



### 4. Model Training

Train your model and evaluate its performance using validation data.

In [15]:
# Write your code here for Model Training here


#define the iteration
import torch.optim as optim
import torch.nn.functional as F

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_fn = nn.CrossEntropyLoss()

# Training loop
def train(model, train_loader, optimizer, loss_fn, num_epochs=20):
    model.train()
    global dl_loss_value  # Declare the global variable for the loss value
    for epoch in range(num_epochs):
        total_loss = 0
        for texts, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(texts)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        dl_loss_value = total_loss / len(train_loader)  # Store the loss value in dl_loss_value
        print(f'Epoch {epoch + 1}, Loss: {dl_loss_value}')

# Train the model
train(model, train_loader, optimizer, loss_fn, num_epochs=5)


#create the training loop


Epoch 1, Loss: 0.3927784115076065
Epoch 2, Loss: 0.16077557663832392
Epoch 3, Loss: 0.08840025863277592
Epoch 4, Loss: 0.04481233745713585
Epoch 5, Loss: 0.030548392448274952


### 5. Model Evaluation
Evaluate your model's performance on the test data using the grading scheme defined above.

In [16]:
# Write your code here for Model Evaluation here


#define the iteration
from sklearn.metrics import accuracy_score

# Evaluation function with dl_accuracy
def evaluate(model, test_loader):
    model.eval()
    global dl_accuracy  # Declare the global variable for accuracy
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for texts, labels in test_loader:
            outputs = model(texts)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())

    dl_accuracy = accuracy_score(all_labels, all_preds) * 100  # Store accuracy in dl_accuracy
    print(f'Test Accuracy: {dl_accuracy:.2f}%')

# Evaluate the model
evaluate(model, test_loader)

#create the training loop


Test Accuracy: 97.13%


## Submission

Once you are satisfied with the performance of your model, then you run the code block below to submit your project.


In [17]:
# Submit Method

# Do not change the code below
question_id = "01_dl_project_accuracy"
submit(student_id, name, assignment_id, str(dl_accuracy), question_id, drive_link)
question_id = "02_dl_project_loss_value"
submit(student_id, name, assignment_id, str(dl_loss_value), question_id, drive_link)

'Assignment successfully submitted'

## FIN